In [1]:
import numpy as np
import pandas as pd
from netCDF4 import Dataset
from matplotlib.backends.backend_pdf import PdfPages
from mce import get_logger
from mce.core.forcing import RfCO2
from mce.core.climate import IrmBase
from mce.util.plot_base import PlotBase
from mce.util.plotfunc import plot_fitting

In [2]:
logger = get_logger(__name__)

In [16]:
class PlotPdf(PlotBase):
    def init_general(self, **kw):
        kw = kw.copy()

        paper_width = 297.
        paper_height = 210.
        mmpi = 25.4

        kw['left'] = 1.5
        kw['top'] = 1.5
        width = kw['height'] * kw['aspect']
        kw['right'] = paper_width/mmpi - (
            2*width + kw['wspace'] + kw['left']
        )
        kw['bottom'] = paper_height/mmpi - (
            2*kw['height'] + kw['hspace'] + kw['top']
        )
        super().init_general(**kw)

    def savefig(self, path, **kw):
        """Wrapper for figure.savefig()
        Default parameters defined by self.kw_savefig and updates
        specified by the keyword arguments are passed

        Parameters
        ----------
        path
            Output file path with supported file name extension,
            such as 'png', 'svg', etc.
        """
        if isinstance(path, PdfPages):
            logger.info('page [{}])'.format(path.get_pagecount() + 1))
            path.savefig(self.figure)
        else:
            kw_savefig = self.kw_savefig.copy()
            kw_savefig.update(kw)
            logger.info(f'saved to {path}')
            self.figure.savefig(path, **kw_savefig)

In [17]:
forcing = RfCO2()
climate = IrmBase()
myplt = PlotPdf()

In [5]:
with pd.HDFStore('data/cmip_normalized.h5', 'r') as store:
    df_norm_cmip5 = store['CMIP5/climate_norm1']
    df_norm_cmip6 = store['CMIP6/climate_norm1']

In [6]:
df_parms = {}

with Dataset('mce/data/parms/parms_irm-3_rtnt-tas_cmip5.nc') as ncf:
    df_parms['CMIP5'] = pd.DataFrame({
        k: v[:].filled()
        for k, v in ncf.variables.items() if k not in ['dataset']
    }, index=[
        ''.join(x.astype(str)).strip()
        for x in ncf.variables['dataset'][:]
    ])

with Dataset('mce/data/parms/parms_irm-3_rtnt-tas_cmip6.nc') as ncf:
    df_parms['CMIP6'] = pd.DataFrame({
        k: v[:].filled()
        for k, v in ncf.variables.items() if k not in ['dataset']
    }, index=[
        ''.join(x.astype(str)).strip()
        for x in ncf.variables['dataset'][:]
    ])

In [7]:
df_parms = pd.concat(df_parms).rename(
    columns={
        'time_constant_0': 'tau0',
        'time_constant_1': 'tau1',
        'time_constant_2': 'tau2',
        'amplitude_0': 'a0',
        'amplitude_1': 'a1',
        'amplitude_2': 'a2',
    }
).rename_axis(['mip', 'dataset']).sort_index(key=lambda x: x.str.lower())

In [12]:
space = {
    'height': 2.5,
    'aspect': 1.4,
    'wspace': 0.9,
    'hspace': 0.4,
    'left': 1.5,
    'top': 1.5,
}
# paper_width = 297.
# paper_height = 210.
# mmpi = 25.4
# width = space['height'] * space['aspect']
# space['right'] = paper_width/mmpi - (
#     2*width + space['wspace'] + space['left'])
# space['bottom'] = paper_height/mmpi - (
#     2*space['height'] + space['hspace'] + space['top'])
# xpos = (space['left'] + width + 0.5*space['wspace']) / (paper_width/mmpi)
# ypos = (
#     space['bottom'] + 2*space['height'] + space['hspace'] + 0.3
# ) / (paper_height/mmpi)

In [13]:
min_n = []
max_n = []
min_t = []
max_t = []
gcm_data = {}

for dataset, parms in df_parms.loc['CMIP6'].iterrows():
    df = df_norm_cmip6.loc[dataset]
    gcm = {
        '4x_n': df.loc[('abrupt-4xCO2', 'rtnt')].dropna().values,
        '4x_t': df.loc[('abrupt-4xCO2', 'tas')].dropna().values,
        '1p_n': df.loc[('1pctCO2', 'rtnt')].dropna().values,
        '1p_t': df.loc[('1pctCO2', 'tas')].dropna().values,
    }
    gcm_data[dataset] = gcm

    min_n.append(min(gcm['4x_n'].min(), gcm['1p_n'].min()))
    max_n.append(max(gcm['4x_n'].max(), gcm['1p_n'].max()))
    min_t.append(min(gcm['4x_t'].min(), gcm['1p_t'].min()))
    max_t.append(max(gcm['4x_t'].max(), gcm['1p_t'].max()))

xlim = [min(min_t)-0.2, max(max_t)]
ylim = [min(min_n)-0.2, max(max_n)]

fmax = (df_parms['alpha'] * df_parms['beta'] * np.log(4.)).max()
fmax_reg = (df_parms['lambda_reg'] * df_parms['ecs_reg'] * 2).max()
tmax = (df_parms['alpha'] * df_parms['beta'] * np.log(4.) \
        / df_parms['lambda']).max()
tmax_reg = df_parms['ecs_reg'].max() * 2

xlim[1] = max(xlim[1], tmax, tmax_reg)
ylim[1] = max(ylim[1], fmax, fmax_reg)

In [23]:
outpath = PdfPages('untracked/fitting_results.pdf')

In [24]:
for dataset, parms in df_parms.loc['CMIP6'].iloc[:3].iterrows():
    parms = df_parms.loc[('CMIP6', dataset)]

    forcing.parms.update(
        alpha=parms['alpha'],
        beta=parms['beta'],
    )
    climate.parms.update(
        asj=parms[['a0', 'a1', 'a2']].values,
        tauj=parms[['tau0', 'tau1', 'tau2']].values,
        lamb=parms['lambda'],
    )

    names = {'var_n': 'rtnt', 'var_t': 'tas', 'dataset': dataset}
    plot_fitting(
        myplt, gcm_data[dataset], forcing, climate, names, parms.to_dict(),
    )
    myplt(2).set_xlim(*xlim)
    myplt(2).set_ylim(*ylim)
    myplt.savefig(outpath)

myplt.clear()

[2024-09-20 18:01:40 __main__] INFO:page [1])
[2024-09-20 18:01:40 __main__] INFO:page [2])
[2024-09-20 18:01:40 __main__] INFO:page [3])


<Figure size 841.89x595.276 with 0 Axes>

<Figure size 841.89x595.276 with 0 Axes>

<Figure size 841.89x595.276 with 0 Axes>

In [25]:
outpath.close()

In [4]:
pdf = PdfPages('../image/fitting_results.pdf')

names = {'var_n': obj.var_n, 'var_t': obj.var_t, 'dataset': ''}

for dataset in cmip_models:
    parms = parms_all.drop('mip', axis=1).loc[dataset]
    forcing, irm, ecs, tcr = obj.get_irm(parms)
    names['dataset'] = dataset
    p1 = plot_fitting(gcm_data[dataset], forcing, irm, names, parms.to_dict(), kw_space=kw_space)
    p1.figure.axes[2].set_xlim(*xlim)
    p1.figure.axes[2].set_ylim(*ylim)
    p1.figure.text(xpos, ypos, dataset, ha='center', va='bottom', fontsize='x-large')
    p1.savefig(pdf)

pdf.close()

[2020-05-24 15:05:19 mce] INFO:page [1])
[2020-05-24 15:05:19 mce] INFO:page [2])
[2020-05-24 15:05:19 mce] INFO:page [3])
[2020-05-24 15:05:20 mce] INFO:page [4])
[2020-05-24 15:05:20 mce] INFO:page [5])
[2020-05-24 15:05:20 mce] INFO:page [6])
[2020-05-24 15:05:21 mce] INFO:page [7])
[2020-05-24 15:05:21 mce] INFO:page [8])
[2020-05-24 15:05:21 mce] INFO:page [9])
[2020-05-24 15:05:22 mce] INFO:page [10])
[2020-05-24 15:05:22 mce] INFO:page [11])
[2020-05-24 15:05:22 mce] INFO:page [12])
[2020-05-24 15:05:23 mce] INFO:page [13])
[2020-05-24 15:05:23 mce] INFO:page [14])
[2020-05-24 15:05:23 mce] INFO:page [15])
[2020-05-24 15:05:23 mce] INFO:page [16])
[2020-05-24 15:05:24 mce] INFO:page [17])
[2020-05-24 15:05:24 mce] INFO:page [18])
[2020-05-24 15:05:24 mce] INFO:page [19])
[2020-05-24 15:05:25 mce] INFO:page [20])
[2020-05-24 15:05:25 mce] INFO:page [21])
[2020-05-24 15:05:25 mce] INFO:page [22])
[2020-05-24 15:05:26 mce] INFO:page [23])
[2020-05-24 15:05:26 mce] INFO:page [24])
[